In [1]:
from collections import defaultdict
from collections import Counter
#this is for the simple mixture BAM file
def parse_tsv_mix(input_file):
    loci_dict={230769616:"D1S1656", 129294244:"D10S1248", 2171088:"Th01", 5983977:"vWA", 12297020:"D12S391", 82148025:"D13S317",86352702:"D16S539", 63281667:"D18S51", 29926235:"D19S433", 1489653:"TPOX", 68011947:"D2S441", 218014859:"D2S1338", 19181973:"D21S11", 37140287:"D22S1045", 45540739:"D3S1358", 154587736:"FGA", 123775556:"D5S818", 150076324:"CSF1PO", 88277144:"SE33", 84160226:"D7S820", 124894865:"D8S1179"}
    allele_dict=defaultdict(list)
    with open(input_file,'r') as infile:
        for line in infile:
            if line.startswith("#"):
                continue
            columns=line.strip().split('\t')
            if columns[9] == 'NA':
                continue
            else:
                #compiling all allele counts for each str loci
                allele_dict[loci_dict[int(columns[1])]].append(float(columns[9]))

        for key in allele_dict.keys(): 
            #filtering for duplicate alleles and allele with single counts
            unfiltered_counts=allele_dict[key]
            counts=Counter(unfiltered_counts)
            filtered_counts=[x for x in unfiltered_counts if counts[x]>1]
            allele_dict[key]=list(set(filtered_counts))

        return allele_dict
            

In [2]:
from collections import defaultdict
from collections import Counter
#this is for the single contributor BAM file. The difference is we are selecting the top 2 allele counts for each loci + allele count appear more than once in tsv file!
def parse_tsv_single(input_file):
    loci_dict={230769616:"D1S1656", 129294244:"D10S1248", 2171088:"Th01", 5983977:"vWA", 12297020:"D12S391", 82148025:"D13S317",86352702:"D16S539", 63281667:"D18S51", 29926235:"D19S433", 1489653:"TPOX", 68011947:"D2S441", 218014859:"D2S1338", 19181973:"D21S11", 37140287:"D22S1045", 45540739:"D3S1358", 154587736:"FGA", 123775556:"D5S818", 150076324:"CSF1PO", 88277144:"SE33", 84160226:"D7S820", 124894865:"D8S1179"}
    allele_dict=defaultdict(list)
    with open(input_file,'r') as infile:
        for line in infile:
            if line.startswith("#"):
                continue
            columns=line.strip().split('\t')
            if columns[9] == 'NA':
                continue
            else:
                #compiling all allele counts for each str loci
                allele_dict[loci_dict[int(columns[1])]].append(float(columns[9]))

        for key in allele_dict.keys(): 
            #filtering for duplicate alleles and allele with single/double counts. Only want top 2 alleles as this is the max in single contributor. 
            unfiltered_counts=allele_dict[key]
            counts=Counter(unfiltered_counts)
            top_two= [allele for allele, count in counts.most_common(2)]
            filtered_counts=[x for x in unfiltered_counts if x in top_two and counts[x]>=2]
            allele_dict[key]=list(set(filtered_counts))

        return allele_dict

In [3]:
MIX_counts=parse_tsv_mix("straglr/MIX_results.tsv")
HG007_counts=parse_tsv_single("straglr/HG_all/HG007_results.tsv")
HG006_counts=parse_tsv_single("straglr/HG_all/HG006_results.tsv")
HG003_counts=parse_tsv_single("straglr/HG_all/HG003_results.tsv")
HG005_counts=parse_tsv_single("straglr/HG_all/HG005_results.tsv")

print(MIX_counts)
print(HG007_counts)

defaultdict(<class 'list'>, {'D1S1656': [16.0, 12.0, 15.0], 'D10S1248': [16.0, 13.0, 13.8, 14.0], 'Th01': [9.8], 'vWA': [16.0, 19.0, 20.0, 14.0], 'D12S391': [17.0, 18.0, 21.0], 'D13S317': [8.0, 10.0, 11.0], 'D16S539': [9.0, 10.0, 11.0], 'D18S51': [12.5, 13.0, 14.0, 13.2, 14.5, 15.0], 'D19S433': [15.0, 16.5, 17.5, 18.2, 17.0, 16.0], 'TPOX': [8.5, 9.5, 11.5], 'D2S441': [11.0, 10.2, 11.8, 12.0], 'D2S1338': [18.0, 19.0, 22.8, 23.0], 'D21S11': [32.8, 34.8, 34.2, 34.0], 'D22S1045': [16.7, 12.7, 15.7], 'D3S1358': [16.0, 19.0, 15.0], 'FGA': [26.0, 22.5, 23.2, 24.8, 24.0, 26.5], 'D5S818': [10.0, 11.0, 13.0], 'CSF1PO': [11.0, 10.0, 11.2, 12.0], 'SE33': [28.5, 18.5, 23.8, 24.5, 28.8], 'D7S820': [10.8, 11.0, 12.0, 14.0], 'D8S1179': [10.8, 12.0, 13.0]})
defaultdict(<class 'list'>, {'D1S1656': [12.0, 15.0], 'D10S1248': [16.0, 13.0], 'Th01': [9.8], 'vWA': [16.0, 19.0], 'D12S391': [21.0, 23.8], 'D13S317': [10.0, 11.0], 'D16S539': [9.0, 11.0], 'D18S51': [15.0, 15.5], 'D19S433': [16.0, 15.0], 'TPOX': [8

In [4]:
#this function is to find if sample (single contributor) is in simple mixture
def contain_itis(sample_dict, mix_dict):
    #a counter if alleles in sample loci are found in mixture
    good_match=0
    all_loci=sample_dict.keys()
    for loci in all_loci:
        if all(allele in mix_dict[loci] for allele in sample_dict[loci]):
            good_match +=1
    print("{}/21 loci matched!".format(good_match))
    #I set the threshold for allele match to be 13 out of a total of 23
    if good_match >= 13:
        print("Sample is likely to be in simple mixture!")
    else:
        print("Sample is unlikely to be in simple mixture!")

        



Limitations: 1) Does not rule out higher order mixtures 2) fail to distinguish true homozygotes from heterozygosity. 

In [5]:
contain_itis(HG007_counts, MIX_counts)

17/21 loci matched!
Sample is likely to be in simple mixture!


Allele frequencies below is based on a US population survey of 1036 people. Data can be found from STRbase

In [6]:
allele_frequencies={'D1S1656': {10:0.00723938223938224, 11:0.0511583011583012, 12:0.0863899613899614, 13:0.0951, 14:0.144787644787645, 14.3:0.00434362934362934, 15:0.161679536679537, 15.3:0.0415057915057915, 16:0.142374517374517, 16.3:0.0680501930501931, 17:0.041988416988417, 17.3:0.10472972972973, 18.0:0.00579150579150579, 18.3:0.0357142857142857, 19.3:0.00916988416988417}, 'D10S1248': {8:0.000965250965250965, 9:0.000965250965250965, 10:0.0028957528957529, 11:0.013030888030888, 12:0.0719111969111969, 13:0.276544401544402, 14:0.295849420849421, 15:0.201254826254826, 16:0.107142857142857, 17:0.0265444015444015, 18:0.00144787644787645, 19:0.00144787644787645}, 'Th01': {5:0.00193050193050193, 6:0.195945945945946, 7:0.29488416988417, 8:0.125482625482625, 9:0.168918918918919, 9.3:0.205598455598456, 10:0.00675675675675676, 11:0.000482625482625483} , 'vWA': {11:0.00144787644787645, 12:0.000965250965250965, 13:0.00337837837837838, 14:0.0955598455598456, 15:0.13465250965251, 16:0.230212355212355, 17:0.262065637065637, 18:0.180019305019305, 19:0.0786679536679537, 20:0.0115830115830116, 21:0.00144787644787645}, 'D12S391': {14:0.000482625482625483, 15:0.0506756756756757, 16:0.0405405405405405, 17:0.124517374517375, 17.1:0.00144787644787645, 17.3:0.0125482625482625, 18:0.208494208494208, 18.1:0.000482625482625483, 18.3:0.013030888030888, 19:0.151544401544402, 19.1:0.00337837837837838, 19.3:0.00482625482625483, 20:0.126447876447876, 20.1:0.000965250965250965, 20.3:0.000482625482625483, 21:0.100868725868726, 22:0.0661196911196911, 22.2:0.000482625482625483, 23:0.0492277992277992, 24:0.0255791505791506, 24.3:0.000482625482625483, 25:0.0115830115830116, 26:0.00337837837837838, 27:0.00241312741312741}, 'D13S317': {8:0.0965250965250965, 9:0.0892857142857143, 10:0.0588803088803089, 11:0.290540540540541, 12:0.305019305019305, 13:0.116312741312741, 14:0.041988416988417, 15:0.00144787644787645}, 'D16S539': {5:0.000482625482625483, 8:0.0212355212355212, 9:0.162644787644788, 10:0.108108108108108, 11:0.291505791505791, 12:0.256756756756757, 13:0.137065637065637, 14:0.0217181467181467, 15:0.000482625482625483}, 'D18S51': {9:0.000965250965250965, 10:0.0048, 11:0.00723938223938224, 12:0.0941119691119691, 13:0.10472972972973, 13.2:0.00144787644787645, 14:0.129343629343629, 14.2:0.000965250965250965, 15:0.166988416988417, 15.2:0.000482625482625483, 16:0.148166023166023, 16.2:0.000482625482625483, 17:0.133204633204633, 18:0.0878378378378378, 19:0.0612934362934363, 20:0.0357142857142857, 21:0.00965250965250965, 21.2:0.000482625482625483, 22:0.00868725868725869, 23:0.00193050193050193, 24:0.000965250965250965, 28:0.000482625482625483}, 'D19S433': {9:0.000482625482625483, 10:0.00434362934362934, 11:0.0260617760617761, 12:0.0834942084942085, 12.2:0.0178571428571429, 13:0.247104247104247, 13.2:0.0318532818532819, 14:0.304054054054054, 14.2:0.0511583011583012, 15:0.117760617760618, 15.2:0.0569498069498069, 16:0.027992277992278, 16.2:0.0231660231660232, 17:0.00241312741312741, 17.2:0.00386100386100386, 18.2:0.00144787644787645}, 'TPOX': {5:0.000483091787439614, 6:0.0318840579710145, 7:0.0072463768115942, 8:0.466183574879227, 9:0.13768115942029, 10:0.0599033816425121, 11:0.244444444444444, 12:0.051207729468599, 13:0.000966183574879227}, 'D2S441': {8:0.000482625482625483, 9:0.00144787644787645, 9.1:0.000965250965250965, 10:0.203185328185328, 11:0.340250965250965, 11.3:0.0487451737451737, 12:0.0994208494208494, 12.3:0.00434362934362934, 13:0.0323359073359073, 13.3:0.000965250965250965, 14:0.226833976833977, 14.3:0.000482625482625483, 15:0.0390926640926641, 16:0.000965250965250965, 17:0.000482625482625483}, 'D2S1338': {15:0.000965250965250965, 16:0.0400579150579151, 17:0.141891891891892, 18:0.0704633204633205, 19:0.148648648648649, 20:0.132722007722008, 21:0.0666023166023166, 22:0.0752895752895753, 23:0.118243243243243, 24:0.097007722007722, 25:0.0834942084942085, 26:0.0226833976833977, 27:0.0019305019305019}, 'D21S11': {24.2:0.000482625482625483, 25.2:0.000482625482625483, 26:0.000482625482625483, 26.2:0.000482625482625483, 27:0.0386100386100386, 28:0.16457528957529, 28.2:0.000482625482625483, 29:0.204150579150579, 29.2:0.00144787644787645, 29.3:0.000482625482625483, 30:0.247586872586873, 30.2:0.0217181467181467, 30.3:0.000965250965250965, 31:0.0801158301158301, 31.2:0.0772200772200772, 32:0.013996138996139, 32.2:0.0912162162162162, 33:0.00434362934362934, 33.1:0.00144787644787645, 33.2:0.0328185328185328, 34:0.00241312741312741, 34.2:0.00144787644787645, 35:0.00772200772200772, 36:0.00337837837837838, 37:0.000965250965250965, 38:0.000482625482625483, 39:0.000482625482625483}, 'D22S1045': {8:0.00241312741312741, 10:0.0168918918918919, 11:0.129826254826255, 12:0.0255791505791506, 13:0.00530888030888031, 14:0.0526061776061776, 15:0.320945945945946, 16:0.297297297297297, 17:0.136583011583012, 18:0.0101351351351351, 19:0.00241312741312741}, 'D3S1358': {11:0.000482625482625483, 12:0.00144787644787645, 13:0.0028957528957529, 14:0.0873552123552124, 15:0.30453667953668, 15.2:0.000482625482625483, 16:0.282818532818533, 17:0.204150579150579, 18:0.105694980694981, 19:0.00916988416988417, 20:0.000965250965250965}, 'FGA': {16.2:0.000482625482625483, 17:0.00144787644787645, 17.2:0.000482625482625483, 18:0.0144787644787645, 18.2:0.00579150579150579, 19:0.0579150579150579, 19.2:0.000965250965250965, 20:0.0883204633204633, 21:0.147200772200772, 21.2:0.00193050193050193, 22:0.197393822393822, 22.2:0.00675675675675676, 22.3:0.000482625482625483, 23:0.155888030888031, 23.2:0.00241312741312741, 24:0.137065637065637, 24.2:0.000965250965250965, 25:0.1003861003861, 25.2:0.000482625482625483, 26:0.0492277992277992, 27:0.0197876447876448, 28:0.00530888030888031, 29:0.00241312741312741, 30:0.000965250965250965, 30.2:0.000482625482625483, 31.2:0.000482625482625483}, 'D5S818': {7:0.0111, 8:0.0197876447876448, 9:0.0463320463320463, 10:0.0777027027027027, 11:0.315637065637066, 12:0.353281853281853, 13:0.163127413127413, 14:0.0111003861003861, 15:0.00193050193050193}, 'CSF1PO': {7:0.0231660231660232, 8:0.0212355212355212, 9:0.0294401544401544, 10:0.232142857142857, 11:0.273648648648649, 12:0.344594594594595, 13:0.0656370656370656, 14:0.00916988416988417, 15:0.000965250965250965}, 'SE33': {6.3:0.000482625482625483, 7:0.000482625482625483, 10.2:0.000482625482625483, 11:0.000482625482625483, 11.2:0.000965250965250965, 12:0.00434362934362934, 12.2:0.00193050193050193, 13:0.0115830115830116, 13.2:0.00193050193050193, 14:0.0318532818532819, 14.2:0.00337837837837838, 15:0.0376447876447876, 15.2:0.00241312741312741, 16:0.0487451737451737, 16.2:0.00193050193050193, 16.3:0.000965250965250965, 17:0.0733590733590734, 17.2:0.000482625482625483, 17.3:0.00193050193050193, 18:0.0955598455598456, 18.3:0.000482625482625483, 19:0.0931467181467181, 19.2:0.0028957528957529, 20:0.0723938223938224, 20.2:0.00675675675675676, 21:0.0333011583011583, 21.2:0.0178571428571429, 22:0.0144787644787645, 22.2:0.0231660231660232, 23:0.00434362934362934, 23.2:0.027992277992278, 24:0.000482625482625483, 24.2:0.0231660231660232, 25.2:0.0352316602316602, 26:0.000482625482625483, 26.2:0.0588803088803089, 27:0.000482625482625483, 27.2:0.0738416988416988, 27.3:0.000482625482625483, 28.2:0.0641891891891892, 28.3:0.000965250965250965, 29.2:0.0468146718146718, 30:0.000482625482625483, 30.2:0.0376447876447876, 31:0.00144787644787645, 31.2:0.0188223938223938, 32:0.000482625482625483, 32.2:0.00916988416988417, 33:0.000965250965250965, 33.2:0.00337837837837838, 34:0.00386100386100386, 34.2:0.000482625482625483, 36:0.000482625482625483}, 'D7S820': {6:0.000482625482625483, 7:0.0164092664092664, 8:0.165540540540541, 8.1:0.000482625482625483, 9:0.121621621621622, 10:0.29488416988417, 10.3:0.000482625482625483, 11:0.234555984555985, 12:0.136100386100386, 13:0.0275096525096525, 14:0.00193050193050193}, 'D8S1179': {8:0.0106177606177606, 9:0.00482625482625483, 10:0.0786679536679537, 11:0.0670849420849421, 12:0.141891891891892, 13:0.268339768339768, 14:0.233590733590734, 15:0.140444015444015, 16:0.0487451737451737, 17:0.00386100386100386, 18:0.00193050193050193}}

In [7]:
from math import prod
def calculate_LR(suspect_counts, MIX_counts):
    #using semi continuous drop-in model. 
    #specifying the drop out and drop in probabilities
    pDO=0.05
    pDI=0.01
    #this average allele frequency is a hard estimate from the US population survey. To be used if an allele count in single origin DNA contributor is not found in allele_frequencies dictionary above.
    avg_allele_frequency=0.0745

    loci_list=list(suspect_counts.keys())

    numerator_likelihood=[]
    denominator_likelihood=[]
    #prob_Hp evaluates the probability of evidence given  that the suspect is in the simple mixture while prob_Hd evaluates the random match probability (using pop allele frequencies). 
    for loci in loci_list:
        prob_Hp=[]
        prob_Hd=[]
        counts=0 #to add 1-pDI when there is no drop in to single-origin DNA contributor sample. 
        for allele in suspect_counts[loci]:
            if allele in MIX_counts[loci]:
                prob_Hp.append(1-pDO)
                counts +=1
            else:
                prob_Hp.append(pDI)
            if allele in allele_frequencies[loci]:
                prob_Hd.append(allele_frequencies[loci][allele])
            else:
                prob_Hd.append(avg_allele_frequency)
        if counts==len(suspect_counts[loci]):
            prob_Hp.append(1-pDI)
        if counts == 0: 
            prob_Hp.append(pDO)

        numerator_likelihood.append(prod(prob_Hp))
        denominator_likelihood.append(prod(prob_Hd))
    
    value=prod(numerator_likelihood)/prod(denominator_likelihood)
    if value >= 1:
        print(f"LR is {value} and it is likely that individual is in the simple mixture")
    
    else:
        print(f"LR is {value} and it is unlikely that individual is in the simple mixture")


                

    
    

In [9]:
calculate_LR(HG005_counts, MIX_counts)

LR is 1.2786588304632e+29 and it is likely that individual is in the simple mixture
